In [ ]:
import os
import tensorflow as tf

import cv2
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt 
import glob
import keras
from keras.utils import normalize
from keras.metrics import MeanIoU

In [ ]:
SIZE_X = 640
SIZE_Y = 640
n_classes = 3 #Number of classes for segmentation

In [ ]:
xpath = "./train/images"

In [ ]:
xfiles = sorted(os.listdir(xpath))

In [ ]:
ypath = "./train/masks.jpg"

In [ ]:
xtrain = []

for i in tqdm(xfiles):
    img_path = os.path.join(xpath, i)
    img = cv2.imread(img_path)
    if img is None:
        print(f"Warning: Failed to load image {img_path}")
        continue  # Skip this image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (640, 640), interpolation=cv2.INTER_LINEAR)
    xtrain.append(img)

xtrain = np.array(xtrain)

In [ ]:
np.shape(xtrain)

In [ ]:
# Get a sorted list of image paths
img_paths = sorted(glob.glob(os.path.join(ypath, "*.jpg")))

ytrain = []

# Iterate through the sorted list of image paths
for mask_path in img_paths:
  mask = cv2.imread(mask_path, 0)
  # mask = mask/255.0
  # mask = mask.astype(np.float32)
  mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST)  #Otherwise ground truth changes due to interpolation
  ytrain.append(mask)

#Convert list to array for machine learning processing
ytrain = np.array(ytrain)

In [ ]:
np.shape(ytrain)

In [ ]:
np.unique(ytrain)

In [ ]:
import random
image_number = random.randint(0, 100)

image = xtrain[image_number]
mask = ytrain[image_number]

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

axes[0].imshow(image, cmap = 'jet')
axes[0].set_title('Image')
axes[0].set_axis_off()

axes[1].imshow(mask, cmap = 'gray')
axes[1].set_title('Mask')
axes[1].set_axis_off()

plt.show()

In [ ]:
#Encode labels... but multi dim array so need to flatten, encode and reshape
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = ytrain.shape
train_masks_reshaped = ytrain.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

np.unique(train_masks_encoded_original_shape)

In [ ]:
train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)

np.shape(xtrain), np.shape(train_masks_input)

In [ ]:
import random
image_number = random.randint(0, 100)

image = xtrain[image_number]
mask = train_masks_input[image_number]

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

axes[0].imshow(image, cmap = 'gray')
axes[0].set_title('Image')
axes[0].set_axis_off()

axes[1].imshow(mask, cmap = 'gray')
axes[1].set_title('Mask')
axes[1].set_axis_off()

plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_do_not_use, y_train, y_do_not_use = train_test_split(xtrain, ytrain, test_size = 0.2, random_state = 0)

In [ ]:
len(X_train), len(X_do_not_use), len(y_train), len(y_do_not_use)

In [ ]:
import numpy as np

# Normalize images by dividing by 255 (assuming they are in range [0, 255])
X_train_normalized = X_train.astype('float32') / 255.0
X_do_not_use_normalized = X_do_not_use.astype('float32') / 255.0

# Optional: Normalize the labels if needed, assuming they are binary (0 or 1)
# No normalization is usually needed for labels if they are already in [0, 1] range
y_train_normalized = y_train.astype('float32')
y_do_not_use_normalized = y_do_not_use.astype('float32')

# Check the lengths
print(len(X_train_normalized), len(y_train_normalized), len(X_do_not_use_normalized), len(y_do_not_use_normalized))


In [ ]:
print("Class values in the dataset are ... ", np.unique(y_train), np.unique(y_train).__len__())  # 0 is the background/few unlabeled

In [ ]:
# Custom Haar Wavelet Layer (downsampling)
class HaarWaveletLayer(layers.Layer):
    def __init__(self, **kwargs):
        super(HaarWaveletLayer, self).__init__(**kwargs)

    def call(self, inputs):
        # Assuming input shape is fixed and known, e.g., (batch_size, height, width, channels)
        height = tf.shape(inputs)[1]
        width = tf.shape(inputs)[2]

        # Ensure that the height and width are divisible by 2
        height = height // 2 * 2
        width = width // 2 * 2

        # Perform Haar wavelet transform (downsampling)
        LL = (inputs[:, :height:2, :width:2, :] + inputs[:, 1:height:2, :width:2, :]) / 2
        LH = (inputs[:, :height:2, :width:2, :] - inputs[:, 1:height:2, :width:2, :]) / 2
        HL = (inputs[:, :height:2, :width:2, :] + inputs[:, :height:2, 1:width:2, :]) / 2
        HH = (inputs[:, :height:2, :width:2, :] - inputs[:, :height:2, 1:width:2, :]) / 2
        
        # Concatenate the four components along the last axis (channels)
        result = tf.concat([LL, LH, HL, HH], axis=-1)
        return LL, LH, HL, HH

# Custom Inverse Haar Wavelet Layer (upsampling)
class InverseHaarWaveletLayer(layers.Layer):
    def __init__(self, **kwargs):
        super(InverseHaarWaveletLayer, self).__init__(**kwargs)

    def call(self, inputs):
        # Split the input tensor into four components
        LL, LH, HL, HH = tf.split(inputs, 4, axis=-1)

        # Reconstruct the image (Inverse Haar Transform)
        x1 = (LL + LH) / 2
        x2 = (LL - LH) / 2
        x3 = (HL + HH) / 2
        x4 = (HL - HH) / 2

        # Concatenate the components back (reconstructing the original image)
        reconstructed = tf.concat([x1, x2, x3, x4], axis=1)  # Concatenate along the second axis
        return reconstructed

# Convolution Block
def conv_block(x, filters, kernel_size=30, activation='relu'):
    x = layers.Conv2D(filters, kernel_size, padding='same', activation=activation)(x)
    return x

# U-Net Model Definition with Haar Wavelet Transform
def unet(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Encoder: Layer 1
    x = inputs
    LL1, LH1, HL1, HH1 = HaarWaveletLayer()(x)  
    x11 = conv_block(LL1, 1)  

    # Encoder: Layer 2
    LL2, LH2, HL2, HH2 = HaarWaveletLayer()(x11)
    x21 = conv_block(LL2, 1) 

    # Encoder: Layer 3
    LL3, LH3, HL3, HH3 = HaarWaveletLayer()(x21)
    x31 = conv_block(LL3, 1) 

    # Encoder: Layer 4
    LL4, LH4, HL4, HH4 = HaarWaveletLayer()(x31)
    x41 = conv_block(LL4, 1)  

    # Bottleneck
    bottleneck = conv_block(x41, 1)
    bottleneck = conv_block(bottleneck, 1)
    
    # Decoder: Layer 4
    concat14 = tf.concat([bottleneck, LH4, HL4, HH4], axis=-1)    
    x14 = InverseHaarWaveletLayer()(concat14)
    x14 = tf.image.resize(x14, size=(80, 80), method=tf.image.ResizeMethod.BILINEAR)
    x14 = conv_block(x14, 1)
    
    # Decoder: Layer 3
    concat13 = tf.concat([x14, LH3, HL3, HH3], axis=-1)
    x13 = InverseHaarWaveletLayer()(concat13)
    x13 = tf.image.resize(x13, size=(160, 160), method=tf.image.ResizeMethod.BILINEAR)
    x13 = conv_block(x13, 1)
    
    # Decoder: Layer 2
    concat12 = tf.concat([x13, LH2, HL2, HH2], axis=-1)
    x12 = InverseHaarWaveletLayer()(concat12)
    x12 = tf.image.resize(x12, size=(320, 320), method=tf.image.ResizeMethod.BILINEAR)
    x12 = conv_block(x12, 1)
        
    # Decoder: Layer 1
    concat11 = tf.concat([x12, LH1, HL1, HH1], axis=-1)
    x1_ = InverseHaarWaveletLayer()(concat11)
    x1_ = tf.image.resize(x1_, size=(640, 640), method=tf.image.ResizeMethod.BILINEAR)
    x1_ = conv_block(x1_, 1)
    
    #print(x1_.shape)
    output = layers.Conv2D(1, (1, 1), activation='linear')(x1_)

    # Define the model
    model = Model(inputs, output)

    return model

model = unet((640, 640, 1)) #640

In [ ]:
model.summary()

In [ ]:
n_classes = 3
activation = 'softmax'

LR = 0.0001
optim = keras.optimizers.Adam(LR)

In [ ]:
print(f"xtrain shape: {xtrain.shape}")  # Should be (num_samples, 640, 640, 1)
print(f"ytrain shape: {ytrain.shape}")  # Should match xtrain


In [ ]:
model.compile(optimizer='adam', loss='mse')

batch_size = 8
epochs = 10
history = model.fit(xtrain, ytrain, batch_size=batch_size, epochs=epochs)

In [ ]:
model.save('unet_dwt_model.keras.h5')

In [ ]:
plt.plot(history.history['loss'])
plt.grid()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.title('Error vs Epoch')
plt.show()

In [ ]:
xtrain[0].shape, np.expand_dims(xtrain[0], axis=2).shape

In [ ]:
y = model.predict(xtrain[:10])

In [ ]:
y[0].shape

In [ ]:
plt.imshow(y[8], cmap='gray')
plt.show()

In [ ]:
import random
image_num = random.randint(0, 100)

image1 = xtrain[image_num]
mask1 = ytrain[image_num]

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

axes[0].imshow(image, cmap = 'gray')
axes[0].set_title('Image')
axes[0].set_axis_off()

axes[1].imshow(mask, cmap = 'gray')
axes[1].set_title('Mask')
axes[1].set_axis_off()

plt.show()